In [ ]:
! pip install huggingface_hub
! git config --global credential.helper store

In [58]:
from huggingface_hub import notebook_login
from huggingface_hub import HfFolder
import pandas as pd



In [ ]:
#enter your API key, you can make one for free on HF
notebook_login()   

In [60]:
from huggingface_hub import InferenceApi

inference = InferenceApi("bigscience/bloom",token=HfFolder.get_token())

In [ ]:
exp_condition_df = pd.read_csv('../experiments/experiment3/experiment3_tasks_collection_essentialism.csv')
exp_condition_df = exp_condition_df.fillna("")
exp_condition_df


In [123]:
def bloom_infer(prompt, seed=2022):
    params = {
        "max_new_tokens": 5,
        "top_k": None,
        "top_p": 0.9,
        "temperature": 0.7,
        "do_sample": True,
        "seed": seed,
        "early_stopping": False,
        "no_repeat_ngram_size": None,
        "num_beams": None,
        "eos_token_id": None,
        "length_penalty": 1.0,
    }
    

    response = inference(prompt, params=params)

    return response

In [ ]:

bloom_response_df = pd.DataFrame(columns=exp_condition_df.columns)

for exp_condition in range(...,...): #range(2): #range[1] means the second row of the csv file, can change to range[2] to get the third row of the csv file. We will want to start with [1] since we ran the first row already.
    for rep_id in range(...): 
      exp_row = exp_condition_df.loc[[exp_condition]]
      exp_row["rep_id"] = rep_id + 1

      prompt = ""
      for scenario_id in range(1, 3): #range(1, 3):  # We have two scenarios.
          scenario_column_name = f"scenario_{scenario_id}"  # f"scenario_{scenario_id}" is scenario column name
   
   
          prompt = ""
          prompt += "\n\nQuestion: " + exp_condition_df[scenario_column_name][exp_condition].strip() + " " + exp_condition_df["question"][exp_condition] + "\nAnswer:" + " I think it is "

          print(prompt, end="")
          response = bloom_infer(prompt, seed=rep_id)
          answer = response[0]["generated_text"][len(prompt):].strip()
          print(answer)
          print("-------------------------------------------------------------------\n")


          bloom_response_column_name = f"bloom_response_scenario_{scenario_id}" 
          exp_row[bloom_response_column_name] = answer

          prompt += answer + "\n"

      bloom_response_df = pd.concat([bloom_response_df, exp_row], axis=0)

bloom_response_df.rep_id = bloom_response_df.rep_id.astype(int)

col = bloom_response_df.pop("rep_id")
bloom_response_df.insert(2, col.name, col)

bloom_response_df



In [ ]:
print("\n==========================\n".join(bloom_response_df["bloom_response_question_1"].tolist()))


In [126]:
bloom_response_df.to_csv("../../data/experiment3/experiment3_bloom_unprocessed/....csv", index=False)